## **Train** a ML model locally, **Deploy** the model in Watson Studio, and perform real-time **Prediction** in Maximo-Monitor


A ML model can be trained and deployed to Watson Studio in several different ways. This notebook is a template for training a model locally using data in Maximo-Monitor, and deploying the model in Watson Studio. At the end of this notebook there is further instructions on how to get started with real-time inference.


If you are new to Watson Machine Learning/Cloud Pak for Data you can refer to these links to learn more
- [Watson Machine Learning Documentation](https://www.ibm.com/docs/en/cloud-paks/cp-data/3.5.0?topic=deploying-managing-models-functions)
- [Jupyter Notebook examples for deploying and using Watson Studio models](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-samples-overview.html?context=cpdaas&audience=wdp)
- This notebook is based on [Watson Studio example notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/deployments/custom_library/Use%20scikitlearn%20and%20custom%20library%20to%20predict%20temperature.ipynb)


**Note** Files required to run this notebook that are not provided in this template are: 
1. monitor-credentials.json. Contains credentials to connect to maximo monitor. Contents explained in section **I. Credentials setup**
2. wml-credentials.json. Contains credentials to connect to watson machine learning API. Contents explained in section **I. Credentials setup**
3. Weather.csv. Contains sample data for training. Contents explained in section **III. Training**

You need to provide your own files with the correct file paths. In the template all required inputs are marked with `user-input-required` comment as a guide to fill in the required fields.

**Pre-requisites**
1. install iotfunctions
2. install any other package/s required to train sepcific ml models

In [ ]:
from ibm_watson_machine_learning import APIClient # watson studio
import json # loading credentials
import pandas as pd
import numpy as np

## I. Credentials setup
You need two sets of credentials
1. credentials for monitor to access maximo-monitor database and monitor apis
2. credentials for wml to access watson studio deployment spaces 

The credentials are saved in json files. In the cell below define the relative path (relative to this notebook) for both these credentials file and load the files in local variables

**Note**: Keep the credentials safe and hidden

In [ ]:
MONITOR_CREDENTIALS_FILE_PATH = './dev_resources/monitor-credentials.json'  
WML_CREDENTIALS_FILE_PATH = './dev_resources/wml-credentials.json' # user-input-required. Set it the path of your wml-credentials file relative to this notebook

**More information for monitor-credentials**

[Monitor documentation](https://www.ibm.com/docs/en/maximo-monitor/8.5.0?topic=monitor-connection-parameters)

In [ ]:
with open(MONITOR_CREDENTIALS_FILE_PATH, 'r') as f: 
    monitor_credentials = json.loads(f.read())

**More information for wml-credentials**

1. [Generate new api key](https://cloud.ibm.com/iam/apikeys) for Watson Machine Learning Services or use a api key generated before

2. Save wml-credentials in a json file

```json
{
    url:"https://us-south.ml.cloud.ibm.com",
    apikey:"xxxx"
}
```
**Note**: Depending on the region of your provisioned service instance, use one of the following as your url:
```
Dallas: https://us-south.ml.cloud.ibm.com
London: https://eu-gb.ml.cloud.ibm.com
Frankfurt: https://eu-de.ml.cloud.ibm.com
Tokyo: https://jp-tok.ml.cloud.ibm.com
```

[More information](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html?context=cpdaas&audience=wdp) about WML Authentication

In [ ]:
with open(WML_CREDENTIALS_FILE_PATH, 'r') as f: 
    wml_credentials = json.loads(f.read())

## II. Connect to Watson Machine Learning APIClient

Connect to the APIClient using the credentials loaded in the previous step. This client is used to 
 1. extract deployment spaces
 2. set the deployment space to the space you want to deploy our model in
 3. extract the software specification that you will use to deploy the model
 4. extract the hardware specification that you will use to deploy the model
 
[WML API Client Documentation](http://ibm-wml-api-pyclient.mybluemix.net)

In [ ]:
client = APIClient(wml_credentials)

### Deployment Space

To deploy a model in WML services you need to connect to a deployment space

1. [Create a deployment space](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-spaces_local.html?context=cpdaas&audience=wdp) if you don't have one yet
2. Set the space id manually using the list generated from `client.spaces.list()`

Note: if `client.spaces.list()` is empty you will need to create a deployment space as specified in Step 1

In [ ]:
client.spaces.list(limit=10) #lists deployment spaces

In [ ]:
client.set.default_space('select-and-set-space-id') # user-input-required. Select a space id from the list generated in the above cell

**Software Specification**

[Learn more about spftware specification and why we need them](https://www.ibm.com/docs/en/cloud-paks/cp-data/3.5.0?topic=overview-specifying-model-type-software-specification)

In [ ]:
client.software_specifications.list(limit=5)

**Hardware Specification**


Hardware specifications determine the CPUs and RAM used during inference. Different specifications allows you to scale the deploylment as needed (as a cost).

[Learn more about CUH costs for different hardware specifications](https://dataplatform.cloud.ibm.com/docs/content/wsj/landings/wml-plans.html?context=cpdaas&audience=wdp)

In [ ]:
client.hardware_specifications.list()

## III. Train Model

[Checkout an article about Machine Learning](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf) if you are interested in getting familiar with some basics.

This notebook can be modified and used for regression or classifiaction tasks. [A short refresher for classification vs regression tasks](https://in.springboard.com/blog/regression-vs-classification-in-machine-learning/)

The template provided will load an entity's data and guide you to train a ML model locally using that data. An entity in Monitor has several database tables that store metrics, dimensions, derived metrics, and alerts. To load these several features we provide a step by step guide.  

**Steps to train a model in a notebook**
1. Load training data from Monitor database (for a specified entity)
2. Prepare dataset for training
3. Create a training pipeline
4. Train model

**III.1 Load training data from Monitor database**

An entity in Monitor has several database tables that store raw metrics, dimensions, and alerts. To load these several features we will peform two database reads. The first read will fetch raw_metrics and dimensions, and the second read will fetch the alerts for the user provided entity type. Both these data streams are store is separate comma-separate-values (.csv) files, and avaiable to manipulate as per your needs


The cells below follow these steps to load training data
1. Connect to the database
2. Specify entity_name (and any other user input)
3. Save the data in a csv to use later

**Note** If you already have data available in acsv file you can skip this section **III.1 Load training data from Monitor database** and jump directly to section **III.2 Prepare data**

The user needs to set the following variable in the cell below:
- (Required) entity_name  # name of entity name you'd like to retrieve data from. This is the same name as displayed on the Monitor UI
- (Optional) start_ts # Fetch data starting from this date/time
- (Optional) end_ts # Fetch data until this date/time

**III.1.1 Connect to the database**

Use the monitor credentials loaded section **I. Credentials setup** to connect to the database

In [ ]:
from iotfunctions.db import Database

db = Database(credentials = monitor_credentials)
db_schema = None #  set if you are not using the default

entity_type = db.get_entity_type(name=entity_name)

In [ ]:
# (Setup to fetch alerts)
alert_table_name = "DM_WIOT_AS_ALERT"
alert_table_timestamp_col = "timestamp"
alert_filters = {"entity_type_id": [entity_type._entity_type_id]}

**III.1.2 Specify entity_name (and any other user input)**

In [ ]:
# (User Input) user-input-required

entity_name = 'shraddha_robot' # user-input-required. Set to entity name you'd like to retrieve data from. This is the same name as displayed on the Monitor UI

start_ts = None # user-input-required. (Optional) Fetch data starting from this date/time. Format 'YYYY-MM-DD-HH.MM.ss.mmmmm'. Set to None to disable

end_ts = None # user-input-required. (Optional) Fetch data until this date/time. Format 'YYYY-MM-DD-HH.MM.ss.mmmmm'. Set to None to disable

In [ ]:
metric_df = entity_type.get_data(start_ts = start_ts, end_ts=end_ts) # fetch metric and dimension data

**loaded data: metris and dimension**

The metrics and dimension data for thhe given entity type is loaded in a dataframe indexed with a (id, evt_timestamp) index. `id` is the deviceid and `evt_timestamp` is the event time. This is the same index scheme that the pipeline uses on it's dataframe

In [ ]:
metric_df.head()

In [ ]:
query, _ = db.query(table_name=alert_table_name, schema=db_schema,timestamp_col=alert_table_timestamp_col, start_ts=start_ts, end_ts=end_ts, filters=alert_filters) #query for alert data
alert_df = db.read_sql_query(sql=query.statement) # fetch alert data

**loaded data: alerts**

Alerts data for the provided entity type is loaded in a dataframe 

In [ ]:
alert_df.head()

In [ ]:
# you can perform further data evaluations and manipulations in this cell

**III.1.3 Save data in csv**

The metrics+dimension and alerts data loaded in previous steps can now be saved into .csv files. Running the cell below will save these into files named **metric_{{entity_name}}.csv**  **alert_{{entity_name}}.csv** respectively. In your environment `{{entity_name}}` is replaces by the user specified value in the load data stage.

Both files are saved in the same directory as this notebook. If you wish to create a separate directory to save data you can do so by apeending the relative directory structure to the filepath specified below. For example, if you place the data in a folder named `data` under the directory that contains this notebook modify the filepath like this: 
```python3
metric_data_filepath = f'./data/metric_{entity_name}.csv'
```

In [ ]:
metric_data_filepath = f'metric_{entity_name}.csv'
alert_data_filepath = f'alert_{entity_name}.csv'

In [ ]:
metric_df.to_csv(metric_data_filepath, index=False,header=True) # save metric and dimension data
alert_df.to_csv(alert_data_filepath, index=False,header=True) # save alert data

**III.2 Prepare data**

Load the data saved earlier and manipulate it further to prepare it for training

**Read data back**

In [ ]:
raw_data_df = pd.read_csv(metric_data_filepath) # assumes you saved data using steps above. If you have pre-save data replace `metric_data_filepath` with the path to that .csv. The path should be relative to this notebook 
raw_data_df.head()

In [ ]:
alert_data_df = pd.read_csv(alert_data_filepath) # assumes you saved data using steps above. If you have pre-save data replace `alert_data` with the path to that .csv. The path should be relative to this notebook 
alert_data_df.head()

**Create a dataset that will be used to train the model**

In this template we will use variables `speed` and `acc` to predict `torque`. Depending on your problem statement you can use the all or part of the avaiable data to define your feature vector as well as the prediction variable/s

In [ ]:
data_df = raw_data_df[['acc', 'speed', 'torque']]

In [ ]:
data_df.head()

In [ ]:
# further data manipulation can be performed in this cell

In [ ]:
from sklearn.model_selection import train_test_split

Y = data_df['torque'] # user-input-required. Specify the target/prediction/classification variable here 
X = data_df[['acc', 'speed']] # user-input-required. Provide your feature vector here

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=143) # create a train-test split

In [ ]:
# Use cell for train, test data verification

**III.3 Create training pipeline**

In this template a sample training pipeline is provided. The training pipeline uses [sklearn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that imlpements data transformation steps followed by an estimator (the model). Using the pipeline is optional.

Learning resources
- [sklearn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
- [sklearn Transformers for text data](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data)
- [sklearn Feature Selection](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html?highlight=selectfrommodel#sklearn.feature_selection.SelectFromModel)
- [sklearn Hyperparameter tuning](https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-search)
- [python packages for machine learning](https://github.com/ml-tooling/best-of-ml-python#time-series-data)

Sample code to build a preprocessor to transform input data. The numeric and categorical data are transformed using separate methods. In the sample code below there is a Pipline with a imputer (to fill in missing data) and a scaler to transform numerical data. Using the code below the categorical features are transformed into one hot encoded vector. Both these steps are combined to build the `preprocessor` module to perform input data transformation. 

Use the learning resources above to learn more about the individual parts of the `ColumnTransformer` module and to learn about the different options for data imputers, scalers, and text data transformers avialable 


```python3
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
    
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
```

`numeric_features` is a list of all numerical feature names. In our example `numerical_features=['acc', 'speed']` <br>
`categorical_features` is a list of all numerical feature names


To create and use relevant features you can use `SelecFromModel` for an additional `feature_selection` stage in the training pipeline

```python3
Pipeline(steps=[('preprocessor', preprocessor),
                 ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False))),
                 ('classification', LinearRegression())])
```

In [ ]:
numeric_features = ['acc', 'speed'] # user-input-required

# make sure we have these coulmns in data
numeric_features = list(set(numeric_features).intersection(set(data_df.columns.to_list()))) # make sure we have these coulmns in data

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer


## Example Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

skl_pipeline = Pipeline(steps=[('preprocessor', preprocessor), # (Optional) preprocessor stage
                               ('classification', LinearRegression())]) # user-input-required estimator/machine learnign model you want to run

**III.4 Train model**

You have an option to run the training pipeline as set up in the cell above or to set up hyperparameter tuning in the cell below and run the search pipeline. The training pipeline runs a single training job while the hyperparameter search runs several jobs to find the best model parameters across the ranges specified in `param_grid`. The search pipeline is more computationally intensive and so it takes longer to run.


**(Optional) Set up hyper paratmeter tuning**

- Parameters of pipelines can be set using `__` separated parameter names
- specify parameters to train in `param_grid`
- speciffy other parameters as needed

Refer to [Tuning hyperparameter](https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-search) for further details

In [ ]:
use_hyperparameter_tuning = True
param_grid = {
    'classification__learning_rate': 0.1 * (10 ** np.arange(0, 3))  
}
scoring = 'roc_auc'
search = RandomizedSearchCV(skl_pipeline, param_grid, n_jobs=-1, scoring=scoring, cv=5)

**Run Training Job**

Depending on factors like the model you picked, if you decided to run hyperparameter training, and the computational power you have to run this notebook, the cell below can take a few minutes to several minutes

In [ ]:
if 'use_hyperparameter_tuning' in globals() and use_hyperparameter_tuning:
    print('Running the Hyperparameter tuning ...')
    search.fit(X_train, y_train)
    model = search
else:
    print('Running training pipeline ...')
    model = skl_pipeline

model.fit(X_train, y_train)
print('Finished training job')

**Run additional validation on model**

In [ ]:
y_pred = model.predict(X_test)
rmse = np.mean((np.round(y_pred) - y_test.values)**2)**0.5
print('RMSE: {}'.format(rmse))

**(Optional) Save a local copy of the model**

In [ ]:
import pickle
# save the model to disk
model_filename = 'monitor_wml_model.sav'
pickle.dump(model, open(model_filename, 'wb'))

**(Optional) Load the local copy of the model**

In [ ]:
model = pickle.load(open(model_filename, 'rb'))

## IV. Deploy Model in WML

In this step you will use the connection to wml python client made in **Step II. Connect to Watson ML API Client**. When deploying the trained mode you need to **a.** set the metadata, **b.** retain the model and, **c.** deploy the model to watson machine learning. 

For **a.** we will set the following metadata
1. model name
2. software speficication. Run the following command to get all software specificatons`client.software_specifications.list()`. Pick the specification best suited for your modeling
3. hardware specification Run the following command to get all hardware specificatons`client.hardware_specifications.list()`. Pick the specification best suited for your modeling
<br>
(We extracted these specifications when we connected to the Watson Studio Client)

In [ ]:
base_sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7") # user-input-required

This code sets up software specs when using only scikit-learn for modeling purposes. For more intricate sotware setup when using custom-packages refer to [Watson ML sample notebooks](https://www.ibm.com/docs/en/cloud-paks/cp-data/3.5.0?topic=apis-machine-learning-python-example-notebooks) documentation

In [ ]:
model_props = {
    client.repository.ModelMetaNames.NAME: "MaxTemp prediction model",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23', # user-input-required
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: base_sw_spec_uid
    
}

**b. Retain model**

To save the trained model you need to specifiy a software specification.
Learn more about available [Software Specicifications](https://dataplatform.cloud.ibm.com/docs/content/wsj/wmls/wmls-deploy-python-types.html?context=cpdaas&audience=wdp) and it's usage

In [ ]:
published_model = client.repository.store_model(model=model, meta_props=model_props)

In [ ]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

**c. Deploy model**

For this deployment we used the hardware specification that requires the least amount of CUHs. The decision to use a different hardware specification can be guided by speed, performance and other variables <br>
[More Information on CUH usage for different hardware specifications](https://dataplatform.cloud.ibm.com/docs/content/wsj/landings/wml-plans.html?context=cpdaas&audience=wdp)

The default hardware specification is S(2CPUs 16GB). We are changing it to XXS(1CPU 8GB)

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of test model",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC : { "id":  "b128f957-581d-46d0-95b6-8af5cd5be580"} # user-input-required
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

**Extract deployment uuid and other information**

We need the depolyment uuid to get predictions.
We set this field in the **wml_auth** constant feild defined in [Monitor documentation](). This feild is used by the InvokeWatsonStudio catalog function to make real-time predictions using the deployed model

In [ ]:
deployment_uid = client.deployments.get_uid(created_deployment)
print(deployment_uid)

In [ ]:
client.deployments.get_details(deployment_uid)

**Note** Information needed for scoring
1. wml_credentials
2. deployment space_id
3. deployment id
4. Feature vector

## (Optional) V. Test predictions using deployed model

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

In [ ]:
scoring_payload = {
    "input_data": [{
        'fields': ['acc', 'speed'],
        'values': [[22, 23]]}]
}

In [ ]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [ ]:
print(json.dumps(predictions, indent=2))

At this point you have a trained model deployed in Watson Machine Learning services. Congratulations!

Follow [Monitor documentation]() for catalog function **InvokeWatsonStudio** to see predictions within the Monitor pipeline using the deployed 